In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install neuralcoref from source
# pip installation is not working for me 
!git clone https://github.com/huggingface/neuralcoref.git
!pip install -r "/content/neuralcoref/requirements.txt"
!pip install -e "/content/neuralcoref"

Cloning into 'neuralcoref'...
remote: Enumerating objects: 772, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 772 (delta 10), reused 16 (delta 7), pack-reused 748
Receiving objects: 100% (772/772), 67.85 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (407/407), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 KB 26.6 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.6
    Uninstalling srsly-2.4.6:
      Successfully uninstalled srsly-2.4.6
  Attempting uninstall: 

In [ ]:
# Downgrade Spacy to 2.3.9 Version first
# The coreference tool We are using is not well-maintained and only support Spcay < 3.0.0
!pip uninstall spacy -y
!pip install spacy==2.3.9

Found existing installation: spacy 2.3.9
Uninstalling spacy-2.3.9:
  Successfully uninstalled spacy-2.3.9
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached spacy-2.3.9-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 2.3.9 which is incompatible.


In [ ]:
# Restart your kernel at this point

In [ ]:
# See spacy and nerualcoref is properly installed
import spacy, neuralcoref
# Check the spcay version
print(spacy.__version__)
# Download the "en_core_web_sm"(you must download again because of the version)
spacy.cli.download("en_core_web_sm")

100%|██████████| 40155833/40155833 [00:04<00:00, 8669062.94B/s] 


2.3.9
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
# Load the dataset
import pandas as pd
dataset = pd.read_csv("/content/drive/MyDrive/Corpus_10k.csv")
dataset.head(5)

,cord_uid,title,abstract,authors,publish_time,corpus
0,bjvyx8u1,Knowledge and Attitude towards COVID-19 Vaccin...,BACKGROUND: It is imperative to ensure optimal...,"Aklil, Mastewal Belayneh; Temesgan, Wubedle Ze...",2022-05-03,Coronavirus disease 2019 (COVID-19) is a new s...
1,vfo59saj,Improving Quality and Efficiency in Pediatric ...,INTRODUCTION: Many children with behavioral he...,"Emerson, Beth L.; Setzer, Erika; Blake, Eileen...",2022-01-21,"Before initiating this work, our pediatric eme..."
2,vvr5x5cn,A cross-country comparison of user experience ...,Autonomous solutions for transportation are em...,"Bellone, Mauro; Ismailogullari, Azat; Kantala,...",2021-03-08,This paper analyses Mobility as a Service (Maa...
3,be43zmo6,ddPCR Reveals SARS-CoV-2 Variants in Florida W...,Wastewater was screened for the presence of fu...,"Gering, E.; Colbert, J.; Schmedes, S.; Duncan,...",2021-04-13,Several SARS-CoV-2 variants of concern have be...
4,drejl99k,A Serological Survey of Infectious Disease in ...,BACKGROUND: Gray wolves (Canis lupus) were rei...,"Almberg, Emily S.; Mech, L. David; Smith, Doug...",2009-09-16,Several high-mortality disease outbreaks among...


In [ ]:
# Filter out too long, too short and problematic corpus
too_long = [index for index, text in enumerate(dataset["corpus"]) if type(text) != str or len(text) < 100 or len(text) > 100000]
dataset = dataset.loc[~dataset.index.isin(too_long)].reset_index(drop=True)

In [ ]:
# A function to removing IEEE, APA in-text references, URLs and non-english characters(but keeping punctuation)
import re
def data_cleaning(corpus):
    pattern = r"\[\d+(,\s*\d+)*\]|\(([A-Za-z]+( [A-Za-z]+)+)\., [0-9]+\)|http://\S+|www.\S+|[^\x00-\x7F]+"
    clean_corpus = re.sub(pattern, "", corpus)
    return clean_corpus

dataset["corpus"] = dataset["corpus"].apply(lambda x: data_cleaning(x))

In [ ]:
# A function to replace the coreference words(e.g. it, he, she) to the real entity
def coreference(sentence):
    coref_text = []
    for token in sentence:
        if token._.in_coref:
            main_mention = token._.coref_clusters[0].main
            if not coref_text or coref_text[-1] not in main_mention.text:
                coref_text.append(main_mention.text)
        else:
            coref_text.append(token.text_with_ws)
    coref_text = "".join(coref_text)
    return coref_text

In [ ]:
# Load the language model
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
# Add neuralcoref into the pipline
neuralcoref.add_to_pipe(nlp)
print('Pipeline components included: ', nlp.pipe_names)
# Process the corpus with the pipeline(truncated parts by parts)
corpus = dataset["corpus"].tolist()[4000:]
for index, corp in enumerate(tqdm(corpus)):
    index += 4000
    corpus_nlp = nlp(corp)
    with open(f"Coreference_4000_end/corpus_coref_{index}.txt", "w") as f:
        f.write(coreference(corpus_nlp))

Pipeline components included:  ['tagger', 'parser', 'ner', 'neuralcoref']


100%|██████████| 5786/5786 [10:28:59<00:00,  6.52s/it]
